TESTING A*

In [2]:
from generate_room import generate_env
from utils import *
import matplotlib.pyplot as plt
import random
from a_star import *

In [3]:
def create_map():  
    
    """
    :return: 2D array of the game map, list of tuples of the floor positions
    """
    
    env = generate_env()
    state = env.reset()
    #env.render()
    #plt.imshow(state['pixel'][:2000, 400:900])
    positions = get_floor_positions(state)
    game_map = state['chars']
    return game_map, positions
#print(positions)

In [4]:

def create_coordinates(game_map, positions):
    
    """
    :param game_map: the map of the game
    :param positions: list of position inside the map
            
    :return: starting poin, random target, random excludeds targets
    """
    
    start = get_player_location(game_map)

    while True:
        target = positions[random.randrange(len(positions))]
        excludeds = [positions[random.randrange(len(positions))], positions[random.randrange(len(positions))]]
        if target not in excludeds:
            break
    #print("Start: ", start, "Target: ", target, "Excludeds: ", excludeds)
    return start, target, excludeds

Comparison between a_star and handson_a_star

In [5]:
from queue import PriorityQueue

def a_star_handson(game_map: np.ndarray, start: Tuple[int, int], target: Tuple[int, int],
                   h: callable, excludeds:List[Tuple[int,int]]) -> List[Tuple[int, int]]:
    # initialize open and close list
    open_list = PriorityQueue()
    close_list = []
    # additional dict which maintains the nodes in the open list for an easier access and check
    support_list = {}

    starting_state_g = 0
    starting_state_h = h(start, target)
    starting_state_f = starting_state_g + starting_state_h

    open_list.put((starting_state_f, (start, starting_state_g)))
    support_list[start] = starting_state_g
    parent = {start: None}

    while not open_list.empty():
        # get the node with lowest f
        _, (current, current_cost) = open_list.get()
        # add the node to the close list
        close_list.append(current)

        if current == target:
            print("Target found!")
            path = build_path(parent, target)
            return path

        for neighbor in get_neighbors(game_map, current, excludeds):
            # check if neighbor in close list, if so continue
            if neighbor in close_list:
                continue
            # compute neighbor g, h and f values
            neighbor_g = 1 + current_cost
            neighbor_h = h(neighbor, target)
            neighbor_f = neighbor_g + neighbor_h
            parent[neighbor] = current
            neighbor_entry = (neighbor_f, (neighbor, neighbor_g))
            # if neighbor in open_list
            if neighbor in support_list.keys():
                # if neighbor_g is greater or equal to the one in the open list, continue
                if neighbor_g >= support_list[neighbor]:
                    continue
            
            # add neighbor to open list and update support_list
            open_list.put(neighbor_entry)
            support_list[neighbor] = neighbor_g

    print("Target node not found!")
    return None

This codes confront on 100 experiments

In [8]:
from contextlib import redirect_stdout
import io

# Your existing code for 'create_map', 'create_coordinates', 'a_star', and 'a_star_handson' functions

count = 0
better = 0
worse = 0
err = 0

for i in range(50):
    #print("Iteration: ", i)
    game_map, positions = create_map()
    start, target, excludeds = create_coordinates(game_map, positions)

    # Use StringIO to capture output
    fake_stdout = io.StringIO()

    with redirect_stdout(fake_stdout):
        path_1 = a_star(game_map, start, target, excludeds)
        path_2 = a_star_handson(game_map, start, target, euclidean_distance, excludeds)

    if path_1 == path_2:
        count += 1
    elif len(path_1) < len(path_2):
        for i in path_1:
            t = chr(game_map[i[1], i[0]])
            if t in ["|", "-"] or t in [excludeds]:
                err += 1
        better += 1
    else:
        worse += 1

print("Same: ", count)
print("Better: ", better)
print("Worse: ", worse)
print("Error: ", err)


Same:  50
Better:  0
Worse:  0
Error:  0
